## Beyond Stoner-Wohlfarth Single-grain easy axis model
This notebook showcases how to train ML surrogate models for micromagnetic simulations, where the external magnetic field H and the anisotropy axis K1 are aligned in z-direction.

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from scripts.analyze_magnetic_data import analyze_magnetic_data

## Training data generation

The training data has been created using micromagnetic simulations with the [Mammos-Mumag package](https://github.com/MaMMoS-project/mammos-mumag). An isolated cubic grain with edge length 50nm has been simulated with aligned external magnetic field (H) and anisotropy axis (K1). One hysteresis loop was computed for each combination of material parameters A, Ms, K. From the hysteresis loops, Hc, Mr and BHmax are computed. The input ranges of the intrinsic properties were set to:

· 7.9×10^4 < Ms[A/m] < 3.9×10⁶ 

· 10⁻¹³ < A [J/m] < 10^-11

· 10⁴ < K1 [J/m³] < 10^7

This is the input data for the ML model, available at data/magnetic_materials.csv. In total 1497 data points were computed.

## Data Pre-Processing

Before model training, the dataset is analyzed. A pearson correlation analysis is performed among all variables as well as training a classifier to identify whether a material is a hard or a soft magnet. 

In [ ]:
# writes stdoud of executed functions to .txt file stored in logs/
import os

os.makedirs('../logs/', exist_ok=True)

In [ ]:
analyze_magnetic_data(data_path='../data/magnetic_materials.csv')

### Model training
The below code runs the machine learning pipeline of pre-processing the data, model training and evaluation.

Pre-processing:

1. Remove rows with negative values in the chosen columns (optional).
2. Convert values from A/m to Tesla for specified columns (optional)
3. Apply a log transform to specified columns, except for the ones in log_exclude_cols.

The pre-processing steps and models to train are specified in the configuration file (config/ml_config_test.yaml). Namely, a linear regression (LR) model, a random forest (RF), the LASSO regression, a Gaussian process and a fully connected neural network (FCNN) are fitted. Note that separate regressors have been trained for the hard and soft magnetic materials. 

In [ ]:
from scripts.train_model import MLPipeline
import argparse

In [ ]:
configpath = '../config/ml_config_test.yaml'

In [ ]:
pipeline = MLPipeline(configpath)

In [ ]:
pipeline.run()

### Plot Metrics
Observed versus fitted plots and tables containing the metric results are generated with the below code snippet and stored under 'results/plots/' and 'results/metric_tables/'.

In [ ]:
from scripts.plot_metrics import *

In [ ]:
results_dir = '../results/'

In [ ]:
output = '../results/metric_tables'

In [ ]:
# Set matplotlib style
plt.style.use('ggplot')

In [ ]:
# Set default output directory as a subdirectory of the input directory
results_path = Path(results_dir)
if output is None:
    output_path = results_path / "metric_tables"
else:
    output_path = Path(output)

In [ ]:
# Create output directory
output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load results
results = load_results(results_dir)

In [ ]:
# Create tables
create_metric_tables(results, output_path)
        
print(f"Tables and plots generated successfully in {output_path}")